# 웹 애플리케이션에 머신 러닝 모델 내장

# ch8에서 학습한 로지스틱 회귀 모델 다시 불러오기

In [1]:
import numpy as np
import re
from nltk.corpus import stopwords

stop = stopwords.words('english')

# 텍스트 데이터 정제. 불용어 제외한 단어 토큰으로 분리
def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower() +
                  ' '.join(emoticons).replace('-', ''))
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

In [2]:
# 한 번에 문서 하나씩 읽어서 반환하는 제너레이터(generator) 함수

def stream_docs(path):
    with open(path, 'r', encoding='utf-8') as csv:
        next(csv)  # 헤더 넘기기
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

In [3]:
# stream_docs 함수 테스트

next(stream_docs(path='movie_data.csv'))

('"In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70\'s, they discover the criminal and a net of power and money to cover the murder.<br /><br />""Murder in Greenwich"" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich f

In [4]:
# stream_docs 함수에서 문서를 읽어 size 매개변수에서 지정한 만큼 문서 반환하는 함수

def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        pass
    return docs, y

In [5]:
# 외부 메모리 학습에 CountVectorizer, TfidfVectorizer 사용 못함
# HashingVectorizer 사용

from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier

vect = HashingVectorizer(decode_error='ignore',
                         n_features=2**21,
                         preprocessor=None,
                         tokenizer=tokenizer)
clf = SGDClassifier(loss='log', random_state=1, max_iter=1)
doc_stream = stream_docs(path='movie_data.csv')

In [6]:
# 외부 메모리 학습 시작

import pyprind

pbar = pyprind.ProgBar(45)
classes = np.array([0, 1])

for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:42


In [7]:
# 모델 성능 평가

X_test, y_test = get_minibatch(doc_stream, size=5000)
X_test = vect.transform(X_test)

print('정확도: %.3f' % clf.score(X_test, y_test))

정확도: 0.868


In [8]:
# test 데이터 활용 문서 사용해서 모델 업데이트

clf = clf.partial_fit(X_test, y_test)

# 학습된 사이킷런 추정기 저장

In [9]:
import pickle
import os

dest = os.path.join('movieclassifier', 'pkl_objects') 

if not os.path.exists(dest):
    os.makedirs(dest)  # 'movieclassifier', 'pkl_objects' directory 생성
    
# 직렬화(serialization)
pickle.dump(stop,  # 대상 객체
            open(os.path.join(dest, 'stopwords.pkl'), 'wb'),  # 파이썬 객체가 저장될 파일 객체
            protocol=4)  # 최신 pickle 프로토콜로 설정
pickle.dump(clf,
            open(os.path.join(dest, 'classifier.pkl'), 'wb'),
            protocol=4)

In [28]:
%%writefile movieclassifier/vectorizer.py
# 아래 스크립트 vectorizer.py 파일로 저장

from sklearn.feature_extraction.text import HashingVectorizer
import re
import os
import pickle

cur_dir = os.path.dirname(__file__)
stop = pickle.load(open(os.path.join(cur_dir,
                                     'pkl_objects',
                                     'stopwords.pkl'), 'rb'))

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower() +
                  ' '.join(emoticons).replace('-', ''))
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

vect = HashingVectorizer(decode_error='ignore',
                         n_features=2**21,
                         preprocessor=None,
                         tokenizer=tokenizer)

Overwriting vectorizer.py


In [11]:
# 파이썬 디렉토리 'movieclassifier'로 변경

import os

os.chdir('movieclassifier')

In [12]:
# vectorizer 임포트하고 분류기 복원 코드 실행

import pickle
import re
import os
from vectorizer import vect

clf = pickle.load(open(os.path.join('pkl_objects',
                                    'classifier.pkl'), 'rb'))

In [13]:
# 복원한 분류기 객체로 문서 샘플 전처리 및 감성 레이블 예측

import numpy as np

label = {0: '양성', 1: '음성'}
example = ['I love this movie']

X = vect.transform(example)

print('예측: %s\n확률: %.2f%%' %\
      (label[clf.predict(X)[0]],
       np.max(clf.predict_proba(X))*100))

예측: 음성
확률: 81.48%


# 데이터를 저장하기 위해 SQLite 데이터베이스 설정

In [14]:
# SQLite 데이터베이스 생성 및 두 개의 영화 리뷰 저장

import sqlite3

conn = sqlite3.connect('reviews.sqlite')  # SQLite 데이터베이스 파일 연결

c = conn.cursor()  # 데이터베이스 커서 생성
c.execute('DROP TABLE IF EXISTS review_db')
c.execute('''CREATE TABLE review_db  
          (review TEXT, sentiment INTEGER, date TEXT)''')  # review_db 테이블 생성. 컬럼 3개.

example1 = 'I love this movie'
c.execute('''INSERT INTO review_db
           (review, sentiment, date) VALUES
           (?, ?, DATETIME('now'))''', (example1, 1))

example2 = 'I disliked this movie'
c.execute('''INSERT INTO review_db
           (review, sentiment, date) VALUES
           (?, ?, DATETIME('now'))''', (example2, 0))

conn.commit()  # 데이터베이스 변경사항 저장
conn.close()  # 데이터베이스와 연결 닫음

In [15]:
# 테이블 저장 확인

conn = sqlite3.connect('reviews.sqlite')

c = conn.cursor()
c.execute('''SELECT * FROM review_db WHERE date
           BETWEEN "2017-01-01 00:00:00" AND DATETIME('now')''')

results = c.fetchall()

conn.close()

print(results)

# SQLite 전용 데이터베이스 브라우저 사용 => 그래픽 인터페이스로 db확인 가능

[('I love this movie', 1, '2020-03-12 07:53:36'), ('I disliked this movie', 0, '2020-03-12 07:53:36')]


# 플라스크(Flask) 웹 애플리케이션 개발

In [16]:
%%writefile ../1st_flask_app_1/app.py
# app.py 파일 생성

from flask import Flask, render_template

app = Flask(__name__)  # 새로운 플라스크 인스턴스 __name__ 으로 초기화

# templates 폴더 아래에 있는 HTML 파일 화면에 출력
@app.route('/')  # 라우트 데코레이터: 특정 URL이 index함수를 실행하도록 지정
def index():
    return render_template('first_app.html')

# 파이썬 인터프리터에 의해 직접 실행될 때만 run 메서드 사용
if __name__ == '__main__':
    app.run()  # 애플리케이션 실행

Overwriting ../1st_flask_app_1/app.py


In [20]:
%%writefile ../1st_flask_app_2/app.py
# 수정된 app.py 파일 생성

from flask import Flask, render_template, request
from wtforms import Form, TextAreaField, validators

app = Flask(__name__)

# 필요한 폼 필드 추가
class HelloForm(Form):  # wtforms.Form 상속
    sayhello = TextAreaField('', [validators.DataRequired()]) 
    
# 시작 웹 페이지에 텍스트 필드 추가    
@app.route('/')
def index():
    form = HelloForm(request.form)  # request.form: 사용자가 폼에 입력한 데이터   
    return render_template('first_app.html', form=form)

# HTML 폼으로 전달된 내용 검증한 후 hello.html 페이지를 출력
@app.route('/hello', methods=['POST'])
def hello():
    form = HelloForm(request.form)
    if request.method == 'POST' and form.validate():  # POST 방식 사용
        name = request.form['sayhello']
        return render_template('hello.html', name=name)
    return render_template('first_app.html', form=form)

# 웹 페이지에서 서버로 데이터를 보내는 방법
# 1. GET 방식: URL 뒤에 파라미터를 붙임
# 2. POST 방식: 전송 메시지 본문에 정보를 실음

if __name__ == '__main__':
    app.run(debug=True)  # 플라스크 디버거 활성화

Overwriting ../1st_flask_app_2/app.py


# 영화 리뷰 분류기를 웹 애플리케이션으로 만들기

In [34]:
%%writefile app.py
# 메인 애플리케이션 app.py 구현

from flask import Flask, render_template, request
from wtforms import Form, TextAreaField, validators
import pickle
import sqlite3
import os
import numpy as np

# 로컬 디렉터리에서 HashingVectorizer를 임포트합니다
from vectorizer import vect

app = Flask(__name__)

######## 분류기 준비
cur_dir = os.path.dirname(__file__)
clf = pickle.load(open(os.path.join(cur_dir, 'pkl_objects', 'classifier.pkl'), 'rb'))
db = os.path.join(cur_dir, 'reviews.sqlite')

def classify(document):
    label = {0: 'negative', 1: 'positive'}
    X = vect.transform([document])
    y = clf.predict(X)[0]
    proba = np.max(clf.predict_proba(X))
    return label[y], proba

def train(document, y):
    X = vect.transform([document])
    clf.partial_fit(X, [y])
    
def sqlite_entry(path, document, y):
    conn = sqlite3.connect(path)
    c = conn.cursor()
    c.execute('''INSERT INTO review_db (review, sentiment, date)
               VALUES (?, ?, DATETIME('now'))''', (document, y))
    conn.commit()
    conn.close()
    
    
######## 플라스크
class ReviewForm(Form):
    moviereview = TextAreaField('', [validators.DataRequired(),
                                     validators.length(min=15)])  # 최소 15글자 이상 입력
    
@app.route('/')
def index():
    form = ReviewForm(request.form)
    return render_template('reviewform.html', form=form)  # reviewform.html 파일로 출력

@app.route('/results', methods=['POST'])
def results():
    form = ReviewForm(request.form)
    if request.method == 'POST' and form.validate():
        review = request.form['moviereview']
        y, proba = classify(review)
        return render_template('results.html',  # results.html 파일에 분류 결과 출력
                               content=review,
                               prediction=y,
                               probability=round(proba*100, 2))
    return render_template('reviewform.html', form=form)

@app.route('/thanks', methods=['POST'])
def feedback():
    feedback = request.form['feedback_button']
    review = request.form['review']
    prediction = request.form['prediction']  # results.html 템풀랏에서 전달된 예측 클래스 레이블
    inv_label = {'negative': 0, 'positive': 1}  # 정수 클래스 레이블로 변환
    y = inv_label[prediction]
    if feedback == 'Incorrect':
        y = int(not(y))
    train(review, y)  # 분류기 업데이트
    sqlite_entry(db, review, y)  # SQLite 데이터베이스에 새로운 레코드로 피드백 추가
    return render_template('thanks.html')  # thanks.html 템플릿 출력

if __name__ == '__main__':
    app.run()  # 테스트 후 debug=True 삭제

Overwriting app.py


In [36]:
%%writefile update.py
# 영화 분류기 업데이트
# SQLite 데이터베이스에 수집된 피드백 데이터를 사용하여 예측 모델 업데이트

import pickle
import sqlite3
import numpy as np
import os

# 로컬 디렉터리에서 HashingVectorizer를 임포트합니다
from vectorizer import vect

def update_model(db_path, model, batch_size=10000):
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    c.execute('SELECT * from review_db')
    
    results = c.fetchmany(batch_size)
    while results:
        data = np.array(result)
        X = data[:, 0]
        y = data[:, 1].astype(int)
        
        classes = np.array([0, 1])
        X_train = vect.transform(X)
        model.partial_fit(X_train, y, classes=classes)
        results = c.fetchmany(batch_size)
        
    conn.close()
    return model

cur_dir = os.path.dirname(__file__)

clf = pickle.load(open(os.path.join(cur_dir, 'pkl_objects', 'classifier.pkl'), 'rb'))
db = os.path.join(cur_dir, 'reviews.sqlite')

# clf = update_model(db_path=db, model=clf, batch_size=10000)

# classifier.pkl 파일을 영구적으로 업데이트하려면
# 다음 코드의 주석을 해제하세요

# pickle.dump(clf, open(os.path.join(cur_dir, 'pkl_objects', 'classifier.pkl'),
#             'wb', protocol=4))

Overwriting update.py


In [37]:
%%writefile app.py
# 메인 애플리케이션 app.py 구현
# update added version

from flask import Flask, render_template, request
from wtforms import Form, TextAreaField, validators
import pickle
import sqlite3
import os
import numpy as np

# 로컬 디렉터리에서 HashingVectorizer를 임포트합니다
from vectorizer import vect

# 로컬 디렉터리에서 업데이트 함수를 임포트합니다
from update import update_model

app = Flask(__name__)

######## 분류기 준비
cur_dir = os.path.dirname(__file__)
clf = pickle.load(open(os.path.join(cur_dir, 'pkl_objects', 'classifier.pkl'), 'rb'))
db = os.path.join(cur_dir, 'reviews.sqlite')

def classify(document):
    label = {0: 'negative', 1: 'positive'}
    X = vect.transform([document])
    y = clf.predict(X)[0]
    proba = np.max(clf.predict_proba(X))
    return label[y], proba

def train(document, y):
    X = vect.transform([document])
    clf.partial_fit(X, [y])
    
def sqlite_entry(path, document, y):
    conn = sqlite3.connect(path)
    c = conn.cursor()
    c.execute('''INSERT INTO review_db (review, sentiment, date)
               VALUES (?, ?, DATETIME('now'))''', (document, y))
    conn.commit()
    conn.close()
    
    
######## 플라스크
class ReviewForm(Form):
    moviereview = TextAreaField('', [validators.DataRequired(),
                                     validators.length(min=15)])  # 최소 15글자 이상 입력
    
@app.route('/')
def index():
    form = ReviewForm(request.form)
    return render_template('reviewform.html', form=form)  # reviewform.html 파일로 출력

@app.route('/results', methods=['POST'])
def results():
    form = ReviewForm(request.form)
    if request.method == 'POST' and form.validate():
        review = request.form['moviereview']
        y, proba = classify(review)
        return render_template('results.html',  # results.html 파일에 분류 결과 출력
                               content=review,
                               prediction=y,
                               probability=round(proba*100, 2))
    return render_template('reviewform.html', form=form)

@app.route('/thanks', methods=['POST'])
def feedback():
    feedback = request.form['feedback_button']
    review = request.form['review']
    prediction = request.form['prediction']  # results.html 템풀랏에서 전달된 예측 클래스 레이블
    inv_label = {'negative': 0, 'positive': 1}  # 정수 클래스 레이블로 변환
    y = inv_label[prediction]
    if feedback == 'Incorrect':
        y = int(not(y))
    train(review, y)  # 분류기 업데이트
    sqlite_entry(db, review, y)  # SQLite 데이터베이스에 새로운 레코드로 피드백 추가
    return render_template('thanks.html')  # thanks.html 템플릿 출력

if __name__ == '__main__':
    clf = update_model(db_path=db,
                       model=clf,
                       batch_size=10000)
    app.run()  # 테스트 후 debug=True 삭제

Overwriting app.py
